# I. NetworkX et convergence de suite

In [ ]:
%matplotlib inline

from pprint import pprint
import random
import numpy as np
from itertools import permutations, combinations

import matplotlib.pyplot as plt
import networkx as nx

### 1) Ecrire une fonction pour génerer un graphe networkX avec x noeuds et y liens aléatoires. Précisez une valeur aléatoire entre 0 et 1 pour chaque noeuds. Cette valeur sera symbolisée comme étant le score par défault du noeud.

In [ ]:
NODES = 2000
EDGES = 5000

In [ ]:
def generate_graph(n_nodes, n_edges):
    # generate nodes dict
    nodes = dict([(str(node+1), np.around(random.random(), 3)) for node in list(range(n_nodes))])
    # generate edges dict with random edges
    edges = {}
    while len(edges) < n_edges:
        pair = [x for x in random.choices([*nodes.keys()], k=2)]
        edges['-'.join(pair)] = nodes[pair[0]]
    # convert dicts to directed networkx graph
    G = nx.DiGraph()
    for couple, proba_to_win in edges.items():
        a, b = couple.split('-')
        G.add_edge(a, b, weight=edges[a + '-' + b])
    return G

G = generate_graph(n_nodes=NODES, n_edges=EDGES)
print(G)

### 2) Afficher ce graph

In [ ]:
def plot_graph(G, node_size=500, display_edge_labels=False):
    fig = plt.figure(figsize=(20, 20))
    edge_labels=dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = node_size)
    nx.draw_networkx_labels(G, pos)
    if display_edge_labels:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    nx.draw_networkx_edges(G, pos, edge_color='r', arrows=True)
    plt.show()

plot_graph(G, display_edge_labels=True)

### 3) ecrite une fonction pour calculer un score pour chaque noeud comme la moyenne des valeurs de ses noeuds entrants

In [ ]:
def compute_score(G, scores):
    for node in G.nodes():
        score_node = scores[node]
        nb_edges_in = len(G.in_edges(node))
        if nb_edges_in > 0:
            new_score = 0
            for edge in G.in_edges(node):
                a = edge[0]
                new_score += scores[a]
            new_score /= float(nb_edges_in)
            scores[node] = new_score
    return scores

### 4) le calcul du score d'un noeuds dépend donc de ses voisins. C'est donc x calcul de scores de noeuds qui correspondent à x suites à faire converger. Faites converger ces scores en précisant une condition d'arrêt (en calculant la différence L1 avec l'état précédent) ainsi qu'un nombre d'itération maximum à ne pas dépasser.

In [ ]:
n_it = 15
eps = 0 # on peut mettre ici 0 car les suites atteignent leurs limites. Autrement il est d'usage de mettre une faible valeur (ex: 1e-10)
N = G.number_of_nodes()

diffs = []
scores = {n: random.random() for n in G.nodes()}
for it in range(n_it):
    prev_scores = scores.copy()
    # recompute scores
    scores = compute_score(G, scores)
    # check convergence
    diff = sum([abs(scores[node] - prev_scores[node]) for node in G.nodes])
    diffs.append(diff)
    print(it, ":", diff)
    if diff <= N*eps:
        print('Iteration converged in %d iterations.' % it)
        break

print(sum(scores.values()))

### 5) reprener les différences entre chaque itérations et afficher les sur un graph, en x les itérations, en y la différence.

In [ ]:
import pandas as pd
pd.DataFrame(diffs, columns=['diff']).plot()

# II. NetworkX centralités et coupes de graphe

### Calcul de mesures de centralité classiques

In [ ]:
# Betweenness centrality
betweenness_scores = nx.betweenness_centrality(G)
print(betweenness_scores)
# Eigenvector centrality
eigenvector_scores = nx.eigenvector_centrality(G)
print(eigenvector_scores)
# Pagerank centrality
pagerank_scores = nx.pagerank(G)
print(pagerank_scores)

In [ ]:
# Ordre de référence des noeuds
node_order = list(G.nodes())

# Construction du DataFrame
df_centrality = (
    pd.DataFrame({
        "betweenness": betweenness_scores,
        "eigenvector": eigenvector_scores,
        "pagerank": pagerank_scores,
    })
    .loc[node_order]
    .reset_index()
    .rename(columns={"index": "id_node"})
)

df_centrality

### Un exemple de coupe de graphe

In [ ]:
# --- Définition des seuils (coupe) ---
seuil_pr = np.percentile(df_centrality["pagerank"], 75)
seuil_bw = np.percentile(df_centrality["betweenness"], 75)

# Indicateur de coupe
df_centrality["selected"] = (
    (df_centrality["pagerank"] >= seuil_pr) &
    (df_centrality["betweenness"] >= seuil_bw)
)

# --- sous-graphe ---
G_cut = G.subgraph(df_centrality.loc[df_centrality["selected"], "id_node"])
print("avant", G.number_of_nodes(), G.number_of_edges())
print("apres", G_cut.number_of_nodes(), G_cut.number_of_edges())

# --- Visualisation ---
plt.figure(figsize=(8, 6))

# Tous les noeuds
plt.scatter(
    df_centrality["pagerank"],
    df_centrality["betweenness"],
    alpha=0.4,
    label="Noeuds"
)

# Noeuds retenus par la coupe
plt.scatter(
    df_centrality.loc[df_centrality["selected"], "pagerank"],
    df_centrality.loc[df_centrality["selected"], "betweenness"],
    s=80,
    label="Noeuds sélectionnés",
)

# Lignes de coupe
plt.axvline(seuil_pr, linestyle="--", label="Seuil PageRank")
plt.axhline(seuil_bw, linestyle="--", label="Seuil Betweenness")

plt.xlabel("PageRank")
plt.ylabel("Betweenness centrality")
plt.title("Coupe de graphe sur deux critères")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
# --- Paramètres de la coupe droite ---
alpha = 50.0        # poids PageRank
beta = 50.0       # poids Betweenness (souvent plus petite → amplifiée)
tau = np.percentile(
    alpha * df_centrality["pagerank"] + beta * df_centrality["betweenness"], 75
)

# Score linéaire
df_centrality["linear_score"] = alpha * df_centrality["pagerank"] + beta * df_centrality["betweenness"]

# Coupe
df_centrality["selected"] = df_centrality["linear_score"] >= tau

# --- Droite de coupe : beta*y = tau - alpha*x ---
x_vals = np.linspace(df_centrality["pagerank"].min(), df_centrality["pagerank"].max(), 200)
y_vals = (tau - alpha * x_vals) / beta

# --- Visualisation ---
plt.figure(figsize=(8, 6))

# Tous les noeuds
plt.scatter(
    df_centrality["pagerank"],
    df_centrality["betweenness"],
    alpha=0.4,
    label="Noeuds"
)

# Noeuds sélectionnés
plt.scatter(
    df_centrality.loc[df_centrality["selected"], "pagerank"],
    df_centrality.loc[df_centrality["selected"], "betweenness"],
    s=80,
    label="Noeuds sélectionnés"
)

# Droite de coupe
plt.plot(
    x_vals,
    y_vals,
    linestyle="--",
    linewidth=2,
    label="Coupe droite"
)

plt.xlabel("PageRank")
plt.ylabel("Betweenness centrality")
plt.title("Coupe droite (frontière linéaire)")
plt.legend()
plt.grid(True)
plt.show()
